In [958]:
from Bio import SeqIO
import urllib.parse as up
import os
import psycopg2
import re
from Bio import Entrez
from collections import Counter
from IAP_seqs import Seq

# Lista Cenas Importantes

Erro mt random a povoar references

Ver parte das sequences --> Precisamos de mais espaço mas o SQL limita isso  Data too long for column 'Translacion_seq' at row 1

Cannot add or update a child row: a foreign key constraint fails (`Base Dados IAP`.`Genbank`, CONSTRAINT `Genbank-references` FOREIGN KEY (`ID`) REFERENCES `References` (`ID`))

Ligar Base de Dados

In [959]:
import mysql.connector as SQLC   #ligação Bruno

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="root",
   password ="login123",
   database = "Base Dados IAP",
   auth_plugin="mysql_native_password"
)
DataBase.autocommit= True
cur = DataBase.cursor()
cur.execute("""
            SET FOREIGN_KEY_CHECKS=0""")

Ficheiro GB 

In [960]:
id_genbank_input = input("ID NCBI")

In [961]:
handle = Entrez.efetch(db='nucleotide',retmode='txt',id=id_genbank_input, rettype = "gb") # dowload do ficheiro
record = SeqIO.read(handle, "gb")

In [962]:
# # file = "/home/sa_bruno/Documentos/GitHub/Projeto_LB/ARL15/ARL15.gb"                
# file = "Teste.gb"
# record = SeqIO.read(file, "genbank") 

# Obter todas as variáveis / Povoar Base de Dados 

# Tabela Locus

In [963]:
# Locus 
ID = record.name
Size = len(record.seq)
Molecular_type = record.annotations["molecule_type"] + " " + record.annotations["topology"]
Genbank_division = record.annotations["data_file_division"] 
Modification_date = record.annotations["date"]

In [964]:
print(ID)
print(Size)
print(Molecular_type)
print(Genbank_division)
print(Modification_date)

V00642
3569
mRNA linear
PHG
21-OCT-1996


In [965]:
try:
        cur.execute(f"""
                INSERT INTO Locus (ID, Size, Molecular_type, Genbank_division, Modification_date)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                    (ID, Size, Molecular_type, Genbank_division, Modification_date))

except:
        print("Erro Povoação Locus")

# Tabela Genbank

Usamos


In [966]:
# Genbank
Organism = record.annotations["organism"]
Accession = record.annotations["accessions"][0]
Definition = record.description
Keywords = ""

for c in record.annotations["keywords"]:
    Keywords += c + ", "
Keywords = Keywords[0: (len(Keywords) -2)]

Sequence = str(record.seq)

In [967]:
print(ID)
print(Organism)
print(Accession)
print(Definition)
print(Keywords)
print(Sequence)

V00642
Escherichia phage MS2
V00642
phage MS2 genome
coat protein, genome
GGGTGGGACCCCTTTCGGGGTCCTGCTCAACTTCCTGTCGAGCTAATGCCATTTTTAATGTCTTTAGCGAGACGCTACCATGGCTATCGCTGTAGGTAGCCGGAATTCCATTCCTAGGAGGTTTGACCTGTGCGAGCTTTTAGTACCCTTGATAGGGAGAACGAGACCTTCGTCCCCTCCGTTCGCGTTTACGCGGACGGTGAGACTGAAGATAACTCATTCTCTTTAAAATATCGTTCGAACTGGACTCCCGGTCGTTTTAACTCGACTGGGGCCAAAACGAAACAGTGGCACTACCCCTCTCCGTATTCACGGGGGGCGTTAAGTGTCACATCGATAGATCAAGGTGCCTACAAGCGAAGTGGGTCATCGTGGGGTCGCCCGTACGAGGAGAAAGCCGGTTTCGGCTTCTCCCTCGACGCACGCTCCTGCTACAGCCTCTTCCCTGTAAGCCAAAACTTGACTTACATCGAAGTGCCGCAGAACGTTGCGAACCGGGCGTCGACCGAAGTCCTGCAAAAGGTCACCCAGGGTAATTTTAACCTTGGTGTTGCTTTAGCAGAGGCCAGGTCGACAGCCTCACAACTCGCGACGCAAACCATTGCGCTCGTGAAGGCGTACACTGCCGCTCGTCGCGGTAATTGGCGCCAGGCGCTCCGCTACCTTGCCCTAAACGAAGATCGAAAGTTTCGATCAAAACACGTGGCCGGCAGGTGGTTGGAGTTGCAGTTCGGTTGGTTACCACTAATGAGTGATATCCAGGGTGCATATGAGATGCTTACGAAGGTTCACCTTCAAGAGTTTCTTCCTATGAGAGCCGTACGTCAGGTCGGTACTAACATCAAGTTAGATGGCCGTCTGTCGTATCCAGCTGCAAACTTCCAGACAACGTGCAACATATCGCGACGTATCGTGATATGGTTTTA

In [968]:
try:
        cur.execute(f"""
                 INSERT INTO Genbank (ID, Organism, Accession, Definition, Keywords)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                        (ID, Organism, Accession, Definition, Keywords))
except:
        print("Erro Povoação Genbank")

# Tabela Sequences  / Parte para Algoritmos

In [969]:
alg_seq = Seq(Sequence).count_nucleotides()

Count_A = alg_seq[0]
Count_C = alg_seq[1]
Count_G = alg_seq[2]
Count_T = alg_seq[3]

In [970]:
print(ID)
print(Sequence)
print(Count_A)
print(Count_C)
print(Count_G)
print(Count_T)

V00642
GGGTGGGACCCCTTTCGGGGTCCTGCTCAACTTCCTGTCGAGCTAATGCCATTTTTAATGTCTTTAGCGAGACGCTACCATGGCTATCGCTGTAGGTAGCCGGAATTCCATTCCTAGGAGGTTTGACCTGTGCGAGCTTTTAGTACCCTTGATAGGGAGAACGAGACCTTCGTCCCCTCCGTTCGCGTTTACGCGGACGGTGAGACTGAAGATAACTCATTCTCTTTAAAATATCGTTCGAACTGGACTCCCGGTCGTTTTAACTCGACTGGGGCCAAAACGAAACAGTGGCACTACCCCTCTCCGTATTCACGGGGGGCGTTAAGTGTCACATCGATAGATCAAGGTGCCTACAAGCGAAGTGGGTCATCGTGGGGTCGCCCGTACGAGGAGAAAGCCGGTTTCGGCTTCTCCCTCGACGCACGCTCCTGCTACAGCCTCTTCCCTGTAAGCCAAAACTTGACTTACATCGAAGTGCCGCAGAACGTTGCGAACCGGGCGTCGACCGAAGTCCTGCAAAAGGTCACCCAGGGTAATTTTAACCTTGGTGTTGCTTTAGCAGAGGCCAGGTCGACAGCCTCACAACTCGCGACGCAAACCATTGCGCTCGTGAAGGCGTACACTGCCGCTCGTCGCGGTAATTGGCGCCAGGCGCTCCGCTACCTTGCCCTAAACGAAGATCGAAAGTTTCGATCAAAACACGTGGCCGGCAGGTGGTTGGAGTTGCAGTTCGGTTGGTTACCACTAATGAGTGATATCCAGGGTGCATATGAGATGCTTACGAAGGTTCACCTTCAAGAGTTTCTTCCTATGAGAGCCGTACGTCAGGTCGGTACTAACATCAAGTTAGATGGCCGTCTGTCGTATCCAGCTGCAAACTTCCAGACAACGTGCAACATATCGCGACGTATCGTGATATGGTTTTACATAAACGATGCACGTTTGGCATGGTTGTCGTCTCTAGGTATCTTGAACCCACTAGGTATAGTGTGG

In [971]:
try:

        cur.execute("""
                INSERT INTO Sequences (ID, Sequence, Count_A, Count_C, Count_T, Count_G)    
                VALUES (%s, %s, %s, %s, %s,%s)
                """,
                        (ID, Sequence, Count_A, Count_C, Count_T, Count_G))   

except:
        print("Erro Povoação Sequences")

# Tabela Features

In [972]:
temp_l = []
for c in range(len(record.features)):
    temp_l.append(record.features[c].type)

y = Counter(temp_l)
Gene = y["gene"]
# Regulatory = 
#(ID, CDS, Gene, Regulatory, exons, poli_A-site, misc_feature, mRNA))      # acabar esta parte dps  easy
# so contar apartir daqui 
  # Counter({'gene': 5125, 'CDS': 4997, 'tRNA': 105, 'rRNA': 22, 'source': 1, 'tmRNA': 1})
CDS_count = y["CDS"]
Regulatory = y["regulatory"]
exons = y["exons"]
poli_A_site = y["poli_A-site"]                    #  poli_A-site
misc_feature = y["misc_feature"]
mRNA = y["mRNA"]
Location_span = str(record.features[0].location)

In [973]:
print(ID)
print(CDS_count)
print(Gene)
print(Regulatory)
print(poli_A_site)
print(misc_feature)
print(mRNA)
print(Location_span)

V00642
4
0
0
0
0
0
[0:3569](+)


In [974]:
try:
        cur.execute("""
        INSERT INTO Features (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 
except:
        print("Erro Features")

Erro Features


In [975]:
cur.execute("""
        INSERT INTO Features (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)    
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
                (ID, CDS_count, Gene, Regulatory, exons, poli_A_site, misc_feature, mRNA, Location_span)) 

# Tabela CDS

In [976]:
# usar para dps
CDS_list = []

ID_cds = []
Translacion_seq = []
Localization = []
ID_protein = []
Protein = []
for c in range(len(record.features)):
    if record.features[c].type == "CDS":
        CDS_list.append(record.features[c])

# Translacion_seq
for c in range(len(CDS_list)):
    ID_cds.append(CDS_list[c])
    Translacion_seq.append(CDS_list[c].qualifiers["translation"])
    Localization.append(CDS_list[c].location)
    ID_protein.append(CDS_list[c].qualifiers["protein_id"])
    try:
        Protein.append(CDS_list[c].qualifiers["product"])
    except:
        Protein.append("------")
    


In [977]:
for c in range(len(ID_cds)):
    print(ID_cds[c])
    print()
    print(Translacion_seq[c])
    print()
    print(Localization[c])
    print()
    print(ID_protein[c])
    print()
    print(Protein[c])
    print()
    print()


type: CDS
location: [129:1311](+)
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GOA:P03610', 'InterPro:IPR005563', 'UniProtKB/Swiss-Prot:P03610']
    Key: note, Value: ['unnamed protein product; A-protein']
    Key: protein_id, Value: ['CAA23988.1']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MRAFSTLDRENETFVPSVRVYADGETEDNSFSLKYRSNWTPGRFNSTGAKTKQWHYPSPYSRGALSVTSIDQGAYKRSGSSWGRPYEEKAGFGFSLDARSCYSLFPVSQNLTYIEVPQNVANRASTEVLQKVTQGNFNLGVALAEARSTASQLATQTIALVKAYTAARRGNWRQALRYLALNEDRKFRSKHVAGRWLELQFGWLPLMSDIQGAYEMLTKVHLQEFLPMRAVRQVGTNIKLDGRLSYPAANFQTTCNISRRIVIWFYINDARLAWLSSLGILNPLGIVWEKVPFSFVVDWLLPVGNMLEGLTAPVGCSYMSGTVTDVITGESIISVDAPYGWTVERQGTAKAQISAMHRGVQSVWPTTGAYVKSPFSMVHTLDALALIRQRLSR']


['MRAFSTLDRENETFVPSVRVYADGETEDNSFSLKYRSNWTPGRFNSTGAKTKQWHYPSPYSRGALSVTSIDQGAYKRSGSSWGRPYEEKAGFGFSLDARSCYSLFPVSQNLTYIEVPQNVANRASTEVLQKVTQGNFNLGVALAEARSTASQLATQTIALVKAYTAARRGNWRQALRYLALNEDRKFRSKHVAGRWLELQFGWLPLMSDIQGAYEMLTKVHLQEFLPMRAVRQVGTNIKLDGRLSYPAANFQTT

In [978]:
num_list = 0                            # cdc ID que falta ver
for c in range(len(ID_cds)):
        cur.execute("""
                INSERT INTO CDS (ID, Translacion_seq, Localization, ID_protein, Protein)    
                VALUES (%s, %s, %s, %s, %s)
                """,
                (ID + f"_{num_list}", str(Translacion_seq[num_list]), str(Localization[num_list]), str(ID_protein[num_list]), str(Protein[num_list]))) 
        num_list += 1

# Tabela References

In [979]:
def get_authors(a):
    exis_5 = re.search(r"authors:\s+([\w\s,\.-]+)", a)          #dúvida (como é que o ciclo sabe se isto está vazio ou não)       
    if exis_5:
        Authors = []
        Authors_work = []
        y = re.search(r"authors:\s+([\w\s,\.-]+)", a)    # isto aqui é o remark
        Authors_work.append(y.group(1))
        # else:
        #     Consortium.append("------")
        help_authors = [] 
        Authors = str(Authors_work[0]).split("., ")
        for i, c in enumerate(Authors):
            f = c.split(" and ")
            help_authors.append([i, f])
        c = len(Authors)
        i = 0
        while c > 0:
            if len(help_authors[i][1]) > 1:
                Authors.remove(Authors[i])
                for o in help_authors[i][1]:
                    Authors.append(o)

            c -= 1
            i += 1
    else:
        Authors = []

    return Authors



In [980]:
# referencias  # faz parte do PubMed INfo
Pubmed = []
Title = []
Journal = []
Consortium = []
Remark = []
Authors = []
b = ""
pubmed_trash = record.annotations["references"]    # resolver isto 
c = len(pubmed_trash)
i = 0
while c > 0:
    a = str(pubmed_trash[i]).replace("\n", "§")   
    
    exist_3 = re.search(r"pubmed id:\s+(\d+)", a)
    if exist_3:
        m = re.search(r"pubmed id:\s+(\d+)", a)                 # COnfirmar PubMEd haver sempre/nao
        Pubmed.append(m.group(1))
    else:
        Pubmed.append("None")

    
    n = re.search(r"title:\s+([\w\s]+)", a)
    Title.append(n.group(1))

    exist_2 = re.search(r"journal:\s+([\w\s.()--,]+)", a)
    if exist_2:
        j = re.search(r"journal:\s+([\w\s.()--,]+)", a)
        Journal.append(j.group(1))
    else:
        Journal.append("None")

    exis = re.search(r"consrtm:\s+([\w\s]+)", a)
    if exis:
        k = re.search(r"consrtm:\s+([\w\s]+)", a)    # isto aqui é o remark
        Consortium.append(k.group(1))
    else:
        Consortium.append("None")         # ter cuidado com isto dps
    

    exis_1 = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\s\d])", a)
    if exis_1:
        l = re.search(r"pubmed id:\s+\d+§comment:\s+([\w\d\s:\[;\(\)\.,!?<>{}\"]+)", a)
        Remark.append(l.group(1))
    else:
        Remark.append("None") 

    Authors.append(get_authors(a))


    
    c -= 1
    i += 1

In [981]:
print(ID)
print(Pubmed)
print(Authors)
print(Title)
print(Journal)
print(Consortium)
print(Remark)

V00642
['5125360', '4555447', '11946702', '809766', '806810', '1264203', 'None', '387256', '2462433', '7934914', 'None']
[['de Wachter,R', 'Merregaert,J', 'Vandenberghe,A', 'Contreras,R.', 'Fiers,W.'], ['Min Jou,W', 'Haegeman,G', 'Ysebaert,M.', 'Fiers,W.'], ['Contreras,R', 'Vandenberghe,A', 'Volckaert,G', 'Min Jou,W.', 'Fiers,W.'], ['Vandenberghe,A', 'Min Jou,W.', 'Fiers,W.'], ['Fiers,W', 'Contreras,R', 'Duerinck,F', 'Haegmean,G', 'Merregaert,J', 'Jou,W.M', 'Raeymakers,A', 'Volckaert,G', 'Ysebaert,M', 'Van de Kerckhove,J', 'Nolf,F.', 'Van Montagu,M.'], ['Fiers,W', 'Contreras,R', 'Duerinck,F', 'Haegeman,G', 'Iserentant,D', 'Merregaert,J', 'Min Jou,W', 'Molemans,F', 'Raeymaekers,A', 'Van den Berghe,A', 'Volckaert,G.', 'Ysebaert,M.'], ['Schwartz,R.M.', 'Dayhoff,M.O.'], ['Beremand,M.N.', 'Blumenthal,T.'], ['Auerswald,E.A', 'Horlein,D', 'Reinhardt,G', 'Schroder,W.', 'Schnabel,E.'], ['de Smit,M.H.', 'van Duin,J.'], ['Min Jou,W.']]
['Studies on the bacteriophage MS2', 'Nucleotide sequence of 

In [982]:
reference_count = 0                  # erro weird 
for c in Pubmed:    
        cur.execute("""
                INSERT INTO References (ID, Pubmed, ID_authors, Title, Journal, Consortium, Remark)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """,
                (ID + f"_{reference_count}", str(Pubmed[reference_count]), str(Authors[reference_count]), str(Title[reference_count]), str(Journal[reference_count]), str(Consortium[reference_count]), str(Remark[reference_count])))
        reference_count += 1

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'References (ID, Pubmed, ID_authors, Title, Journal, Consortium, Remark)
        ' at line 1

# Tabela Authors


In [983]:
for c in Authors:
        for i, a in enumerate(c):
                cur.execute("""
        INSERT INTO Authors (ID_authors, Names)    
        VALUES (%s, %s)
        """,
                (str(c) + f"_{i}", str(a)))

# Tabela Pubmed_Info

In [984]:
Abstract = []                        # mudar para cada referenias 
DOI = []
Affiliation = []
for c in Pubmed:
    if c != "None":
        Entrez.email = 'sa.bruno.2001@gmail.com'
        # handle = Entrez.efetch(db='pubmed',retmode='xml',id="15496913")
        handle = Entrez.efetch(db='pubmed',retmode='xml',id=c)
        resultado = Entrez.read(handle)
        q = str(resultado["PubmedArticle"][0])
        
        if re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q):
            o = re.search(r"AbstractText.:\s+\['([\w\s,\.\(\)%-:]+)", q)
            Abstract.append(o.group(1))
        else:
            Abstract.append("None")
        
        if re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q):
            p = re.search(r"'pubmed'}.,\s+StringElement.'([\d\w\./]+)" , q)
            DOI.append(p.group(1))
        else:
            DOI.append("None")

        if re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q):
            t = re.search(r"'Affiliation':\s+'([,\s\d\w\./]+)" , q)
            Affiliation.append(t.group(1))
        else:
            Affiliation.append("None")
    if c == "None":
            Abstract.append("None")
            DOI.append("None")
            Affiliation.append("None")



In [985]:
print(Pubmed)
print(Title)
print(Abstract)
print(DOI)


['5125360', '4555447', '11946702', '809766', '806810', '1264203', 'None', '387256', '2462433', '7934914', 'None']
['Studies on the bacteriophage MS2', 'Nucleotide sequence of the gene coding for the bacteriophage MS2 coat protein', 'Studies on the bacteriophage MS2', '3', 'A', 'Complete nucleotide sequence of bacteriophage MS2 RNA', 'Genome Nucleic Acids', 'Overlapping genes in RNA phage', 'Expression', 'Translational initiation at the coat', 'Direct Submission']
['None', 'None', 'None', 'None', 'None', "Bacteriophage MS2 RNA is 3,569 nucleotides long. The nucleotide sequence has been established for the third and last gene, which codes for the replicase protein. A secondary structure model has also been proposed. Biological properties, such as ribosome binding and codon interactions can now be discussed on a molecular basis. As the sequences for the other regions of this RNA have been published already, the complete, primary chemical structure of a viral genome has now been establishe

In [986]:
# try:
#         pubmed_count= 0
#         for c in Pubmed:
#                 cur.execute("""
#                         INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI)   
#                         VALUES (%s, %s, %s, %s)
#                         """,
#                         (Pubmed[pubmed_count] + f"_{pubmed_count}", Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count]))   
#         if c != "None":
#                 pubmed_count += 1
# except:
#         print("Erro Povoar Pubmed")

In [987]:
pubmed_count= 0
for c in Pubmed :
        if c != "None":
                cur.execute("""
                        INSERT INTO Pubmed_Info (ID_pubmed, Title, Abstract, DOI, Affiliation)   
                        VALUES (%s, %s, %s, %s, %s)
                        """,
                        (Pubmed[pubmed_count], Title[pubmed_count], Abstract[pubmed_count], DOI[pubmed_count], Affiliation[pubmed_count]))  
                pubmed_count += 1 
        else:
                pubmed_count += 1

# Lixo para baixo

In [988]:
#     cur.execute("""
#         INSERT INTO Authors (ID_authors, Affiliation)    
#         VALUES (%s, %s, %s))
#         """,
#         (Authors, Affiliation))    